In [2]:
import pygame
import random, time, sys, h5py
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from skimage.transform import resize

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
hf = h5py.File('../data.h5', 'r')
frames = torch.tensor(torch.from_numpy(hf["frame"][:]), dtype=torch.float) # (batch_size, 76, 119)
moves = torch.tensor(torch.from_numpy(hf["move"][:]), dtype=torch.long)

C:\Users\arthu\AppData\Local\Temp\ipykernel_9184\2340934927.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frames = torch.tensor(torch.from_numpy(hf["frame"][:]), dtype=torch.float) # (batch_size, 76, 119)
C:\Users\arthu\AppData\Local\Temp\ipykernel_9184\2340934927.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  moves = torch.tensor(torch.from_numpy(hf["move"][:]), dtype=torch.long)


In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=(24, 30), stride=2)
        self.conv2 = nn.Conv2d(3, 1, kernel_size=(10, 10), stride=1)
        self.fc1 = nn.Linear(522, 228) 
        self.fc2 = nn.Linear(228, 753) 

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, start_dim=1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
model = Model()
o = model.forward(frames[:2].unsqueeze(1))
print(o.shape)

torch.Size([2, 753])


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i in range(0, len(frames), batch_size):
        inputs = frames[i:i+batch_size].unsqueeze(1)
        labels = moves[i:i+batch_size].squeeze(1)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(frames):.4f}")


0
8
16
24
32
40
48
56
64
Epoch [1/10], Loss: 0.2386
0
8
16
24
32
40
48
56
64
Epoch [2/10], Loss: 0.2250
0
8
16
24
32
40
48
56
64
Epoch [3/10], Loss: 0.2177
0
8
16
24
32
40
48
56
64
Epoch [4/10], Loss: 0.2129
0
8
16
24
32
40
48
56
64
Epoch [5/10], Loss: 0.2094
0
8
16
24
32
40
48
56
64
Epoch [6/10], Loss: 0.2068
0
8
16
24
32
40
48
56
64
Epoch [7/10], Loss: 0.2048
0
8
16
24
32
40
48
56
64
Epoch [8/10], Loss: 0.2031
0
8
16
24
32
40
48
56
64
Epoch [9/10], Loss: 0.2017
0
8
16
24
32
40
48
56
64
Epoch [10/10], Loss: 0.2007
